In [5]:
import numpy as np
import pandas as pd
import json
import requests
from statistics import median

In [6]:
ballot_df = pd.read_csv('data/rf6_synthetic_ballots.csv')  # Replace 'data.csv' with your actual file path
step_df = ballot_df[ballot_df['Status'].eq('SUBMITTED')]
df = step_df[step_df['Badgeholder'].eq(True)]

update_df = df

json_data = update_df.to_json(orient='records')

parsed_json = json.loads(json_data)
pretty_data = json.dumps(parsed_json, indent=4)

data = parsed_json
num_raw_ballots = len(data)
print(num_raw_ballots)
print(data[0])

10
{'Address': '0x75cac0ceb8a39ddb4942a83ad2aafaf0c2a3e13f', 'Badgeholder': True, 'Status': 'SUBMITTED', 'Category allocations': '[{"ETHEREUM_CORE_CONTRIBUTIONS": "40.637911089076184"}, {"OP_STACK_TOOLING": "51.90051231201454"}, {"OP_STACK_RESEARCH_AND_DEVELOPMENT": "7.46157659890927"}]', 'Payload': '{"budget": 4300000, "category_allocations": [{"GOVERNANCE_ANALYTICS": "40.637911089076184"}, {"GOVERNANCE_INFRA_AND_TOOLING": "51.90051231201454"}, {"GOVERNANCE_LEADERSHIP": "7.46157659890927"}], "project_allocations": [{"0x82ef80ecf81b0bab897fe9ab595544db44b14708993f91c5f5f457f869d2af1d": "4.424924295926856"}, {"0xbdc75c7f2d9d4f797174656b47ef920cc968b82a1235efec4a37816fc8077ef9": "1.8753422491329352"}, {"0xdda0798a5eb762366cd5aa5de87f042567946b47f5cf1d776077a2b00d75c575": "2.403649278914726"}, {"0xd7f9140cd24d5332f81c9a12d5d7da80f0c61f69db015073727064f0b975c875": "0.0"}, {"0xf615a763fa6599d40e52ee2e15ec77f208324c5863a157cdeb243639936476e8": "3.2517630434700244"}, {"0x383672f18d21daf99befe

In [7]:
url = "https://optimism.easscan.org/graphql"
headers = {
    'Content-Type': 'application/json'
}

attestations = pd.DataFrame()

schemaID = "0x41513aa7b99bfea09d389c74aacedaeb13c28fb748569e9e2400109cbe284ee5"

# for ref_uid in filtered_median_scores['application_id']:
    # Updated query to filter by refUID and schemaID
attest_data = {
    "query": f"""
        query Attestations {{
            attestations(where: {{
                schemaId: {{ equals: "{schemaID}" }},

            }}) {{
                id
                attester
                recipient
                refUID
                decodedDataJson
            }}
        }}
    """,
    "variables": {}
}

# Send the request using POST and json parameter
response = requests.post(url, headers=headers, json=attest_data)
dataset = response.json()

attest_df = pd.json_normalize(dataset['data']['attestations'])
voting_group = []
round = []
for i in attest_df['decodedDataJson']:
  dataset = json.loads(i)
  voting_group.append(dataset[3]['value']['value'])
  round.append(dataset[1]['value']['value'])

attest_df['voting_group'] = voting_group
attest_df['round'] = round

def group_naming(group):
    match group:
        case 'B':
            return 'GOVERNANCE_INFRA_AND_TOOLING'
        case 'C':
            return 'GOVERNANCE_LEADERSHIP'
        case 'A':
            return 'GOVERNANCE_ANALYTICS'

attest_df['category_assignment'] = attest_df['voting_group'].apply(group_naming)
attest_df = attest_df.loc[(attest_df['round'] == str(6))]
# attest_df.to_csv(f"rpgf6_category_assignment_{group_type}.csv", encoding='utf-8')
attest_df.to_csv(f"rpgf6_category_assignment.csv", encoding='utf-8')

In [8]:
### Calculate Budget allocation

budget_allocation =[]

addresses = []
# Collect budgets based on category assignment
for entry in data:

    if entry['Address'].lower() not in attest_df['recipient'].str.lower().unique():
      continue

    addresses.append(entry['Address'])
    entry_payload = json.loads(entry["Payload"])
    budget = entry_payload["budget"]
    budget_allocation.append(budget)

budget_median = np.median(budget_allocation)

print(budget_median)

7250000.0


In [9]:
confirmation_df = pd.DataFrame()
confirmation_df['confirmed_addresses'] = addresses
confirmation_df['budget_allocation'] = budget_allocation

confirmation_df.to_csv(f'rpgf6_output_check.csv', index=False)

In [10]:
### Calculate Category allocation

# 1. Isolate the category budget votes: Each badgeholder will have voted on how to allocate OP to all categories (e.g. [Category1: 33%; Category2: 33%; Category3: 34%])
# 2. Calculate the median of Category allocation
# 3. Adjust category allocations to match 100%

category_scores = {
    'GOVERNANCE_INFRA_AND_TOOLING': [],
    'GOVERNANCE_LEADERSHIP': [],
    'GOVERNANCE_ANALYTICS': []
}

addresses = []
category_allocation_detail = []
project_allocation_detail = []
for entry in data:

    if entry['Address'].lower() not in attest_df['recipient'].str.lower().unique():
      print(entry['Address'])
      continue

    addresses.append(entry['Address'].lower())
    entry_payload = json.loads(entry["Payload"])
    allocations = entry_payload["category_allocations"]
    category_allocation_detail.append(entry_payload["category_allocations"])
    project_allocation_detail.append(entry_payload["project_allocations"])
    for item in allocations:
      for category, score in item.items():
          category_scores[category].append(float(score))

ballot_details = pd.DataFrame(
    {'confirmed_addresses': addresses,
     'category_allocation': category_allocation_detail,
     'project_allocation': project_allocation_detail,
    })
# Calculate the median for each category
category_medians = {category: np.median(scores) for category, scores in category_scores.items()}

# Normalize the category medians
total_median = sum(category_medians.values())
normalized_category_medians = {category: (median / total_median) * 100 for category, median in category_medians.items()}

# Output the normalized category allocations
for category, score in normalized_category_medians.items():
    print(f"{category}: {score}")

0x5c30f1273158318d3dc8ffcf991421f69fd3b77d
0xaef766ce8047a11cbb0f8264dea7559fd0b48444
0x19f8a76474ebf9effb269ec5c2b935a3611d6779
0x961aa96febee5465149a0787b03bfa14d8e9033f
GOVERNANCE_INFRA_AND_TOOLING: 38.460656367632765
GOVERNANCE_LEADERSHIP: 35.0424177561356
GOVERNANCE_ANALYTICS: 26.496925876231636


In [11]:
### Calculate Project Scores

# 1. Isolate the Project votes: Each badgeholder will have voted by submitting percentages reflecting the allocation of OP to the projects within a category (e.g. [Project1: 10; Project2: 4;….]).
# 2. Remove Project votes with a COI: For each badgeholder, they will have a number of `null` votes. These should not be considered for results calculation
# 3. Calculate the median scores of projects
# 4. Adjust scores to match 100%
# 5. Repeat step 1 - 4 for all 3 categories

# Calculate median project allocation under each category
project_allocations_by_category = {
    'GOVERNANCE_INFRA_AND_TOOLING': {},
    'GOVERNANCE_LEADERSHIP': {},
    'GOVERNANCE_ANALYTICS': {}
}

accepted_entries = []
# Collect project allocations based on category assignment
for entry in data:
    filtered_df = attest_df.loc[((attest_df['recipient'].str.lower() == entry['Address'].lower()) & (attest_df['round'] == str(6)))]

    entry_update = json.loads(entry["Payload"])

    if not filtered_df.empty:

      entry_update = json.loads(entry["Payload"])

      category = filtered_df.iloc[0]["category_assignment"]
      project_allocations = entry_update["project_allocations"]
      accepted_entries.append(entry['Address'])

      for item in project_allocations:
        for project_id, allocation in item.items():
            if allocation is not None:  # Exclude None values
                if project_id not in project_allocations_by_category[category]:
                    project_allocations_by_category[category][project_id] = []
                project_allocations_by_category[category][project_id].append(allocation)
    else:
      print(entry['Address'])

# Calculate median project allocation for each project under each category
median_project_allocations = {}
for category, projects in project_allocations_by_category.items():
    median_project_allocations[category] = {}

    for project_id, allocations in projects.items():
        median_project_allocations[category][project_id] = median(map(float, allocations))

normalized_project_scores = {}

for category, scores in median_project_allocations.items():
    total_score = sum(scores.values())
    normalized_project_scores[category] = {project_id: (score / total_score) * 100 for project_id, score in scores.items()}

# Output the normalized project scores
for category, scores in normalized_project_scores.items():
    print(f"\n{category}:")
    print(f"no. of projects: {len(scores)}")
    for project_id, normalized_score in scores.items():
        print(f"Project ID: {project_id}, Normalized Score: {normalized_score:.2f}%")

print(f"\n\nAccepted total of {len(accepted_entries)} ballots (out of {num_raw_ballots} submitted).")

0x5c30f1273158318d3dc8ffcf991421f69fd3b77d
0xaef766ce8047a11cbb0f8264dea7559fd0b48444
0x19f8a76474ebf9effb269ec5c2b935a3611d6779
0x961aa96febee5465149a0787b03bfa14d8e9033f

GOVERNANCE_INFRA_AND_TOOLING:
no. of projects: 83
Project ID: 0x82ef80ecf81b0bab897fe9ab595544db44b14708993f91c5f5f457f869d2af1d, Normalized Score: 2.11%
Project ID: 0xbdc75c7f2d9d4f797174656b47ef920cc968b82a1235efec4a37816fc8077ef9, Normalized Score: 0.89%
Project ID: 0xdda0798a5eb762366cd5aa5de87f042567946b47f5cf1d776077a2b00d75c575, Normalized Score: 1.14%
Project ID: 0xd7f9140cd24d5332f81c9a12d5d7da80f0c61f69db015073727064f0b975c875, Normalized Score: 0.00%
Project ID: 0xf615a763fa6599d40e52ee2e15ec77f208324c5863a157cdeb243639936476e8, Normalized Score: 1.55%
Project ID: 0x73deb2a75792f7c688dc87a3080ce18637b7d241ad2b551272fb21aee72d6bf7, Normalized Score: 2.39%
Project ID: 0x762c363ac739a057eb4d414fd17916a2b098a00c7ddeae32d021f6df572561e1, Normalized Score: 0.74%
Project ID: 0x56cb9a566631cf918d682a40877cb4ffe7d

In [12]:
for category, scores in normalized_project_scores.items():
  score_total = normalized_category_medians[category]
  print(f"{category}: {score_total:.2f}%")
  for project_id, normalized_score in scores.items():
    print(f"Project ID: {project_id}, Normalized Score: {(normalized_score/100)*(score_total/100)}")

GOVERNANCE_INFRA_AND_TOOLING: 38.46%
Project ID: 0x82ef80ecf81b0bab897fe9ab595544db44b14708993f91c5f5f457f869d2af1d, Normalized Score: 0.008098380278291451
Project ID: 0xbdc75c7f2d9d4f797174656b47ef920cc968b82a1235efec4a37816fc8077ef9, Normalized Score: 0.00343220215075877
Project ID: 0xdda0798a5eb762366cd5aa5de87f042567946b47f5cf1d776077a2b00d75c575, Normalized Score: 0.00439909580695214
Project ID: 0xd7f9140cd24d5332f81c9a12d5d7da80f0c61f69db015073727064f0b975c875, Normalized Score: 0.0
Project ID: 0xf615a763fa6599d40e52ee2e15ec77f208324c5863a157cdeb243639936476e8, Normalized Score: 0.00595129135320012
Project ID: 0x73deb2a75792f7c688dc87a3080ce18637b7d241ad2b551272fb21aee72d6bf7, Normalized Score: 0.009189118983216133
Project ID: 0x762c363ac739a057eb4d414fd17916a2b098a00c7ddeae32d021f6df572561e1, Normalized Score: 0.0028346676040007443
Project ID: 0x56cb9a566631cf918d682a40877cb4ffe7dfc4d3cbec30f5cc9a1a71c8083618, Normalized Score: 0.009879457560864831
Project ID: 0x0af93b34c4c04748

In [13]:

### Calculate Results (weighted proportional distribution)

# 1. Multiply Project Scores with category scores - done
# 2. Adjust scores to match 100% (there should be no need to readjust) - done
# 3. Implement Max scores and redistribute excess
# 4. Implement min and redistribute excess (while not breaking max rule)

# Constants
total_amount = budget_median
max_amount = total_amount*0.125
# max_amount = 1000000
min_amount = 1000

# Step 1: Calculate total allocation for each category
category_total_allocations = {category: (allocation / 100) * total_amount for category, allocation in normalized_category_medians.items()}

# Final allocations dictionary
final_allocations = {}

# Step 2: Sort projects within each category
for category, scores in normalized_project_scores.items():
    normalized_project_scores[category] = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

# Step 3: Calculate project allocations using weighted proportional distribution
for category, scores in normalized_project_scores.items():
    total_allocation = category_total_allocations[category]
    project_allocations = {}

    # Calculate total score for the category
    total_score = sum(scores.values())

    # Calculate weighted proportional allocations
    for project, score in scores.items():
        allocation = (score / total_score) * total_allocation
        project_allocations[project] = allocation

    # Step 4: Remove projects below min_amount
    project_allocations = {project: allocation for project, allocation in project_allocations.items() if allocation >= min_amount}

    # Step 5: Normalize allocations
    total_allocated = sum(project_allocations.values())

    if total_allocated > 0:
        normalized_allocations = {project: (allocation / total_allocated) * total_allocation for project, allocation in project_allocations.items()}
    else:
        normalized_allocations = {}

    # Step 6: Cap allocations at max_amount
    normalized_allocations = {project: min(allocation, max_amount) for project, allocation in normalized_allocations.items()}

    # Store the final allocations for the category
    final_allocations[category] = normalized_allocations

# Display the final allocations
for category, allocations in final_allocations.items():
    print(f"Allocations for {category}:")
    for project, allocation in allocations.items():
        print(f"  Project {project}: {allocation:,.2f}")


Allocations for GOVERNANCE_INFRA_AND_TOOLING:
  Project 0xb7599398abc3c25cd4564477d85184306982ab595b6b5dc09458e0c449036c41: 79,663.40
  Project 0x48e15f1a280e543983e8e44ac5278aa9a7c2ae10b5ce47063aeaac6e946e87b9: 77,672.54
  Project 0xcdebeb77a8d4ebbf16aff97d72dcea3d080ae73e7848d858c2f051eb2d2aed2c: 76,822.14
  Project 0x879316e11edcc53ed462d391d737aa13c71af8b5ba53be4f1b0dd908cedccaa0: 75,085.46
  Project 0x56cb9a566631cf918d682a40877cb4ffe7dfc4d3cbec30f5cc9a1a71c8083618: 71,626.07
  Project 0xe950477ad2f79bdbf93133a3b7d0984f25b4fbea78c103de8c09e767d84a7191: 69,547.11
  Project 0x6621d15bdec0eb8a9007f6a0cb83ca672860b7c6ed80faf817659c090009ed50: 69,323.30
  Project 0x73deb2a75792f7c688dc87a3080ce18637b7d241ad2b551272fb21aee72d6bf7: 66,621.11
  Project 0xe07d57e306683c8da55db7e36be5e80b7167f140d7c967f6a888861ed0109b12: 65,574.40
  Project 0x8e6becb19c948b0631fd4f054a283eb0a413a2cdf29e9827e33af02aa762018c: 65,535.71
  Project 0x933019d3568e65ef5e798992024ae4b6342f8492fd14b35c0ac0d1bf1a9530

In [14]:
import csv

csv_file_name = f'rpgf6_allocations_weighted.csv'

# Open the CSV file for writing
with open(csv_file_name, mode='w', newline='') as csv_file:
    # Create a CSV writer object
    writer = csv.writer(csv_file)

    # Write the header row
    writer.writerow(['Category', 'Project', 'Allocation'])

    # Write the allocations data
    for category, allocations in final_allocations.items():
        for project, allocation in allocations.items():
            writer.writerow([category, project, f"{allocation:,.2f}"])

print(f"Allocations have been written to {csv_file_name}")

Allocations have been written to rpgf6_allocations_weighted.csv


In [15]:
coi_list = []
for entry in ballot_details['project_allocation']:

  # List to store keys with null values
  null_keys = []

  # Loop through each dictionary in the field
  for dictionary in entry:
      for key, value in dictionary.items():
          # Check if the value is None (null)
          if value is None:
              null_keys.append(key)

  coi_list.append(null_keys)

ballot_details['COI'] = coi_list
ballot_details

,confirmed_addresses,category_allocation,project_allocation,COI
0,0x75cac0ceb8a39ddb4942a83ad2aafaf0c2a3e13f,[{'GOVERNANCE_ANALYTICS': '40.637911089076184'...,[{'0x82ef80ecf81b0bab897fe9ab595544db44b147089...,[0x383672f18d21daf99befe6f41d201f357f70f1883da...
1,0x490c91f38ec57e3ab00811e0c51a62bfed7e81f4,[{'GOVERNANCE_ANALYTICS': '35.58811836201511'}...,[{'0x82ef80ecf81b0bab897fe9ab595544db44b147089...,[0xf615a763fa6599d40e52ee2e15ec77f208324c5863a...
2,0xad4f365a550835d40dc2e95fdffa1e4edd3fbe14,[{'GOVERNANCE_ANALYTICS': '22.11004148078579'}...,[{'0x62bbb21c83167236f1224b33e62174e32053c7646...,[0x615bcadb648c632d3bad75c741e9df2a5ed642b7972...
3,0xa3eac0016f6581ac34768c0d4b99ddcd88071c3c,[{'GOVERNANCE_ANALYTICS': '13.433408206870103'...,[{'0x62bbb21c83167236f1224b33e62174e32053c7646...,[0xf76a71fa1d734d49095648fb8c12c0e3c5734d3ae0c...
4,0x55aed0ce035883626e536254dda2f23a5b5d977f,[{'GOVERNANCE_ANALYTICS': '14.52154581145641'}...,[{'0x62bbb21c83167236f1224b33e62174e32053c7646...,[0xb7599398abc3c25cd4564477d85184306982ab595b6...
5,0x894aa5f1e45454677a8560dde3b45cb5c427ef92,[{'GOVERNANCE_ANALYTICS': '39.65863453815261'}...,[{'0x73a3fdd603c160c48bb61dfdfd6c93f6d8ad3420e...,[0xf0b3c2c47e7b2c56c10e8bc10a2243a082212a4903d...


In [16]:
# Ballot check
recipient_df = attest_df[['recipient',  'voting_group', 'round','category_assignment', 'decodedDataJson']]
recipient_df['confirmed_addresses'] = recipient_df['recipient'].str.lower()
recipient_df = recipient_df.loc[((recipient_df['round'] == str(6)))]
print(df.head())
confirmation_df['confirmed_addresses'] = confirmation_df['confirmed_addresses'].str.lower()
ballot_details['confirmed_addresses'] = ballot_details['confirmed_addresses'].str.lower()

output_checks = pd.merge(confirmation_df,recipient_df,how='left',on=['confirmed_addresses'])
output_checks = pd.merge(output_checks, ballot_details,how='left',on=['confirmed_addresses'])

output_checks.to_csv(f'rpgf6_ballot_details_check.csv')

                                      Address  Badgeholder     Status  \
0  0x75cac0ceb8a39ddb4942a83ad2aafaf0c2a3e13f         True  SUBMITTED   
1  0x490c91f38ec57e3ab00811e0c51a62bfed7e81f4         True  SUBMITTED   
2  0x5c30f1273158318d3dc8ffcf991421f69fd3b77d         True  SUBMITTED   
3  0xaef766ce8047a11cbb0f8264dea7559fd0b48444         True  SUBMITTED   
4  0x19f8a76474ebf9effb269ec5c2b935a3611d6779         True  SUBMITTED   

                                Category allocations  \
0  [{"ETHEREUM_CORE_CONTRIBUTIONS": "40.637911089...   
1  [{"ETHEREUM_CORE_CONTRIBUTIONS": "35.588118362...   
2  [{"ETHEREUM_CORE_CONTRIBUTIONS": "30.412160783...   
3  [{"ETHEREUM_CORE_CONTRIBUTIONS": "6.9274653626...   
4  [{"ETHEREUM_CORE_CONTRIBUTIONS": "13.259005145...   

                                             Payload  
0  {"budget": 4300000, "category_allocations": [{...  
1  {"budget": 2200000, "category_allocations": [{...  
2  {"budget": 650000, "category_allocations":  [{...  
3  {

<ipython-input-16-f0e051442fa5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipient_df['confirmed_addresses'] = recipient_df['recipient'].str.lower()
